In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.text import one_hot
import tensorflow.keras.utils as ku 
import numpy as np
import pandas as pd

In [29]:
data = open('test1.txt',encoding="utf8").read()
data

'Fishy Business\nMan\'s got bigger fish \nTo fry,\nLeaving the smaller fish \nTo Die,\nStill always wondering\nSadness, why.\nIf livin\'s easy,\nDying\'s an escape.\nIf being normal is easy,\nPut on the cape.\nIf being fat is easy,\nShed all the shape.\nIf speaking is easy,\nLay down the tape.\n\nThe One Who Was Meant To Be-\n\nWhen I became we\nNothing seemed difficult \nThe world was indeed made of rainbows\nOf unicorns and extra crispy french fries\nYou made me complete\nYet somehow the darkness never left\nFor the wounds had never healed\nGrey hid behind the curtain of colors\nWanting to crush the horn of hope\nI wanted to be me again\nFor the inverted M was never meant to be.\nYou took my heart and put it right\nIt was but my mind dwelling on the sorrows of past\nMy broken teeth, my swollen soul\nFight had already boarded the flight\nThe one overbooked with my emotions\nIt was never supposed to fly again\nYet you were the pilot of my dreams\nMy defence against destiny\nI hope to b

In [22]:
corpus = data.lower().split("\n")
vocab=20000
token = Tokenizer(num_words=vocab,filters='')
token.fit_on_texts(corpus)
input_seq = []
total = len(token.word_index)+1

In [27]:
for line in corpus:
	seq = token.texts_to_sequences([line])[0]
	for i in range(1, len(seq)):
		n_gram_seq = seq[:i+1]
		input_seq.append(n_gram_seq)
input_seq

[[490, 282],
 [491, 107],
 [491, 107, 194],
 [491, 107, 194, 283],
 [2, 492],
 [284, 1],
 [284, 1, 493],
 [284, 1, 493, 283],
 [2, 494],
 [94, 85],
 [94, 85, 495],
 [496, 497],
 [47, 498],
 [47, 498, 77],
 [499, 48],
 [499, 48, 500],
 [47, 108],
 [47, 108, 501],
 [47, 108, 501, 10],
 [47, 108, 501, 10, 77],
 [195, 13],
 [195, 13, 1],
 [195, 13, 1, 502],
 [47, 108],
 [47, 108, 503],
 [47, 108, 503, 10],
 [47, 108, 503, 10, 77],
 [504, 26],
 [504, 26, 1],
 [504, 26, 1, 505],
 [47, 506],
 [47, 506, 10],
 [47, 506, 10, 77],
 [196, 109],
 [196, 109, 1],
 [196, 109, 1, 507],
 [1, 17],
 [1, 17, 40],
 [1, 17, 40, 14],
 [1, 17, 40, 14, 95],
 [1, 17, 40, 14, 95, 2],
 [1, 17, 40, 14, 95, 2, 508],
 [23, 6],
 [23, 6, 138],
 [23, 6, 138, 8],
 [139, 285],
 [139, 285, 140],
 [1, 32],
 [1, 32, 14],
 [1, 32, 14, 509],
 [1, 32, 14, 509, 49],
 [1, 32, 14, 509, 49, 5],
 [1, 32, 14, 509, 49, 5, 510],
 [5, 511],
 [5, 511, 7],
 [5, 511, 7, 512],
 [5, 511, 7, 512, 513],
 [5, 511, 7, 512, 513, 514],
 [5, 511, 7

In [30]:
max_seq_len = max([len(x) for x in input_seq])
input_seq = np.array(pad_sequences(input_seq, maxlen=max_seq_len, padding='pre'))
input_seq

array([[   0,    0,    0, ...,    0,  490,  282],
       [   0,    0,    0, ...,    0,  491,  107],
       [   0,    0,    0, ...,  491,  107,  194],
       ...,
       [   0,    0,    0, ...,  317,    2, 1892],
       [   0,    0,    0, ...,    2, 1892,    3],
       [   0,    0,    0, ..., 1892,    3,  488]], dtype=int32)

In [31]:
predictor = input_seq[:,:-1]
label = input_seq[:,-1]
label = ku.to_categorical(label,num_classes=total,dtype = "int32")

In [32]:
model = Sequential()
model.add(Embedding(total, 100, input_length=max_seq_len-1))
model.add(Bidirectional(LSTM(150, return_sequences = True)))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dense(total/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 11, 100)           189300    
_________________________________________________________________
bidirectional (Bidirectional (None, 11, 300)           301200    
_________________________________________________________________
dropout (Dropout)            (None, 11, 300)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense (Dense)                (None, 946)               95546     
_________________________________________________________________
dense_1 (Dense)              (None, 1893)              1792671   
Total params: 2,539,117
Trainable params: 2,539,117
Non-trainable params: 0
______________________________________________

In [34]:
apply = model.fit(predictor, label, epochs=200, verbose=1)

Epoch 1/200
363/363 [==============================] - 16s 43ms/step - loss: 6.7724 - accuracy: 0.0538
Epoch 2/200
363/363 [==============================] - 16s 45ms/step - loss: 6.0906 - accuracy: 0.0604
Epoch 3/200
363/363 [==============================] - 16s 44ms/step - loss: 5.8140 - accuracy: 0.0755
Epoch 4/200
363/363 [==============================] - 20s 56ms/step - loss: 5.5916 - accuracy: 0.0903
Epoch 5/200
363/363 [==============================] - 24s 66ms/step - loss: 5.3763 - accuracy: 0.0994
Epoch 6/200
363/363 [==============================] - 26s 72ms/step - loss: 5.1704 - accuracy: 0.1050
Epoch 7/200
363/363 [==============================] - 25s 68ms/step - loss: 4.9844 - accuracy: 0.1100
Epoch 8/200
363/363 [==============================] - 23s 62ms/step - loss: 4.8110 - accuracy: 0.1186
Epoch 9/200
363/363 [==============================] - 24s 67ms/step - loss: 4.6421 - accuracy: 0.1324
Epoch 10/200
363/363 [==============================] - 17s 48ms/step - l

363/363 [==============================] - 14s 39ms/step - loss: 0.8991 - accuracy: 0.8004
Epoch 80/200
363/363 [==============================] - 14s 39ms/step - loss: 0.8960 - accuracy: 0.8013
Epoch 81/200
363/363 [==============================] - 15s 42ms/step - loss: 0.8902 - accuracy: 0.8014
Epoch 82/200
363/363 [==============================] - 14s 39ms/step - loss: 0.8828 - accuracy: 0.8008
Epoch 83/200
363/363 [==============================] - 14s 38ms/step - loss: 0.8666 - accuracy: 0.8058
Epoch 84/200
363/363 [==============================] - 14s 39ms/step - loss: 0.8670 - accuracy: 0.8055
Epoch 85/200
363/363 [==============================] - 14s 39ms/step - loss: 0.8621 - accuracy: 0.8046
Epoch 86/200
363/363 [==============================] - 14s 39ms/step - loss: 0.8593 - accuracy: 0.8043
Epoch 87/200
363/363 [==============================] - 14s 39ms/step - loss: 0.8549 - accuracy: 0.8057
Epoch 88/200
363/363 [==============================] - 14s 39ms/step - loss:

363/363 [==============================] - 14s 40ms/step - loss: 0.6910 - accuracy: 0.8146
Epoch 158/200
363/363 [==============================] - 14s 40ms/step - loss: 0.6920 - accuracy: 0.8140
Epoch 159/200
363/363 [==============================] - 14s 40ms/step - loss: 0.6882 - accuracy: 0.8143
Epoch 160/200
363/363 [==============================] - 14s 40ms/step - loss: 0.6843 - accuracy: 0.8161
Epoch 161/200
363/363 [==============================] - 14s 40ms/step - loss: 0.6891 - accuracy: 0.8123
Epoch 162/200
363/363 [==============================] - 14s 40ms/step - loss: 0.6874 - accuracy: 0.8137
Epoch 163/200
363/363 [==============================] - 16s 43ms/step - loss: 0.6854 - accuracy: 0.8132
Epoch 164/200
363/363 [==============================] - 16s 44ms/step - loss: 0.6800 - accuracy: 0.8137
Epoch 165/200
363/363 [==============================] - 15s 42ms/step - loss: 0.6791 - accuracy: 0.8164
Epoch 166/200
363/363 [==============================] - 15s 40ms/ste

In [37]:
seed_text = "What is this life"
next_words = 50
  
for _ in range(next_words):
	seq = token.texts_to_sequences([seed_text])[0]
	seq = pad_sequences([seq], maxlen=max_seq_len-1, padding='pre')
	predicted = model.predict_classes(seq, verbose=0)
	output_word = ""
	for word, index in token.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)

What is this life to worth the chase. people? will be well day hold. meant to be. real life restored. fever came it over the sorrows of letters life of day day day of day day in smile of hold. hold. someone in smile of gray, meant of hold. hold. it was hold. hold.
